In [ ]:
path,update,s4,target,max_probs,targets=best_physical_path_strategy_(Gp,Gs,0,S,random.choice(targets),C)
nx.draw(s4, with_labels=True)
path=path[0]
print('Path init',path)
trim=copy.deepcopy(path)
path_buff=copy.deepcopy(path)
extra_sensors=[]
while not nx.is_connected(s4):
    #comps=[list(i) for i in nx.connected_components(s4)]
    Gpp=Gp.copy()
    for previous, current in zip(path, path[1:]):
        #print(comps[i])
        #common_values1= union_intersect(S,comps[i],path)
        #common_values2= union_intersect(S,comps[i+1],path)
        #print(common_values1)
        #print(update)
        candidates_inter=find_sensors(update, previous, current)
        mult=0
        print(candidates_inter,'holders')
        if not candidates_inter:
            #candidates=find_sensors(S, previous, current)
            #print(candidates,'new candidates')
            
            #desired_subgraph=set.union(set(previous),candidate_bridges,set(current))
            #print(desired_subgraph)
            subg = Gpp.copy()
            #print(subg)
            subg1=subg.copy()
            #print(nx.has_path(subg, trim[point], trim[point+1]),'Before removal')
            if subg.has_edge(previous,current):
                subg.remove_edge(previous,current)
           # print(subg.has_edge(1,35)
            print('Breakpoints',previous,current)
            mult=900
            set_initial=set()
            set_final=set()
            for sensor in C[previous]:
                for node in S[sensor]:
                    set_initial.add(node)
            for sensor in C[current]:
                for node in S[sensor]:
                    set_final.add(node)
            to_be_removed=[]
            for node in subg.nodes():
                if (max_probs[node][0] == target and node not in path) or (node in targets) or (node in path and node not in (previous, current)):
                    to_be_removed.append(node)
            print('Candidate Bridge nodes',set_initial.intersection(set_final))
            bridge=list(set_initial.intersection(set_final))
            if bridge:
                bridge=[node for node in bridge if node not in to_be_removed]
            if bridge:
                to_be_removed=[]
                for node in subg.nodes():
                    if (max_probs[node][0] == target and node not in path) or (node in targets) or (node in path and node not in (previous, current)):
                        to_be_removed.append(node)
                for node in to_be_removed:
                        if any([nx.has_path(subg,previous,pont) for pont in bridge]):
                            if any([nx.has_path(subg,pont,current) for pont in bridge]):
                            #  print('Node removed 1',node,'Path exists',(subg, trim[point],trim[point+1]))
                                dummy_sub=subg.copy()
                                dummy_sub.remove_node(node)
                                if any([nx.has_path(dummy_sub,previous,pont) for pont in bridge]):
                                    if any([nx.has_path(dummy_sub,pont,current) for pont in bridge]):
                                        subg.remove_node(node)
                                        if node in bridge:
                                            bridge.remove(node)
                               # print('Node removed',node,'Path exists',(subg, trim[point],trim[point+1]))
                print('Retained bridge nodes',bridge)
                p1=nx.shortest_path(subg,previous,bridge[0])
                p2=nx.shortest_path(subg,bridge[0],current)
                p=p1
                p.extend(p2[1:])
                print('Bridge nodes p',p)
            else:
                set_initial=set()
                set_final=set()
                for neighbor in subg.neighbors(previous):
                    for sensor in C[neighbor]:
                        for node in S[sensor]:
                            if node  not in path and node not in (previous, current):
                                set_initial.add(node)
                for neighbor in subg.neighbors(current):
                    for sensor in C[neighbor]:
                        for node in S[sensor]:
                            if node  not in path and node not in (previous, current):
                                set_final.add(node)
                bridge=list(set_initial.intersection(set_final))
                bridge=[node for node in bridge if node not in to_be_removed][0]
                p1=nx.shortest_path(subg,previous,bridge)
                p2=nx.shortest_path(subg,bridge,current)
                p=p1
                p.extend(p2[1:])
                print('Bridge nodes extended',p)
                '''if nx.has_path(subg, previous, current):
                    #print(subg,"removed still paths")
                    mult=1
                    
                    # print('Prior path',nx.shortest_path(subg,trim[point],trim[point+1]))
                    for node in to_be_removed:
                        if nx.has_path(subg,previous,current):
                          #  print('Node removed 1',node,'Path exists',(subg, trim[point],trim[point+1]))
                            dummy_sub=subg.copy()
                            dummy_sub.remove_node(node)
                            if nx.has_path(dummy_sub,previous,current):
                                subg.remove_node(node)
                               # print('Node removed',node,'Path exists',(subg, trim[point],trim[point+1]))
                    subg1=subg.copy()
                
                p=nx.shortest_path(subg1,previous,current)
                print(p)'''
            mult=max(1,len(set(p) & set(to_be_removed)),mult)
            cost=len(p)*mult
            #print(p)
            pos_start=trim.index(previous)
            pos_end=trim.index(current)
            initial=trim[:pos_start]
            final=trim[pos_end+1:]
            path_buff=initial
            path_buff.extend(p)
            path_buff.extend(final)
            trim=copy.deepcopy(path_buff)
            print(trim)
            adjusted_mapping=copy.deepcopy(S)
    
            for sensor,mapping in adjusted_mapping.items():
                new_map=set()
                for node in mapping:
                    if node in trim:
                        new_map.add(node)
                
                adjusted_mapping[sensor] = new_map
            #for extra in extra_sensors:
             #   adjusted_mapping3[extra] = S[extra]
            adjusted_mapping = {k: v for k, v in adjusted_mapping.items() if v}
            s4=create_sensor_graph(adjusted_mapping)
            update=adjusted_mapping
                    
    print('Initial path',path)
    print('Final path',trim)
    #print('Added cover',extra_sensors)
    path=copy.deepcopy(path_buff)
    trim=copy.deepcopy(path_buff)
    print('True path',nx.is_path(Gp,trim),mult)
                
            
    adjusted_mapping=copy.deepcopy(S)
    
    for sensor,mapping in adjusted_mapping.items():
        new_map=set()
        for node in mapping:
            if node in trim:
                new_map.add(node)
        
        adjusted_mapping[sensor] = new_map
    #for extra in extra_sensors:
     #   adjusted_mapping3[extra] = S[extra]
    adjusted_mapping = {k: v for k, v in adjusted_mapping.items() if v}
    s4=create_sensor_graph(adjusted_mapping)
    #print(adjusted_mapping)
    #nx.draw(subgraph3, with_labels=True)
    #print(path_buff)
    #print(adjusted_mapping3)  
    print(adjusted_mapping)
            
    